In [37]:
import pandas as pd
import plotly.express as pe

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,precision_score,recall_score

import numpy as np
from math import log, sqrt

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

## step 1: load the data from the source

In [38]:
credit_df = pd.read_csv('https://query.data.world/s/7af5b2edd2xhj3i5cehwtoy742adto')

### step 2: data preprocessing & exploration

In [39]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 32 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  284807 non-null  int64  
 1   Time        284807 non-null  float64
 2   V1          284807 non-null  float64
 3   V2          284807 non-null  float64
 4   V3          284807 non-null  float64
 5   V4          284807 non-null  float64
 6   V5          284807 non-null  float64
 7   V6          284807 non-null  float64
 8   V7          284807 non-null  float64
 9   V8          284807 non-null  float64
 10  V9          284807 non-null  float64
 11  V10         284807 non-null  float64
 12  V11         284807 non-null  float64
 13  V12         284807 non-null  float64
 14  V13         284807 non-null  float64
 15  V14         284807 non-null  float64
 16  V15         284807 non-null  float64
 17  V16         284807 non-null  float64
 18  V17         284807 non-null  float64
 19  V1

In [40]:
display(credit_df)

,Unnamed: 0,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,2,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,3,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,4,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,5,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,284803,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,284804,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,284805,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,284806,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [41]:
#Let's round off values for easy interpretation
credit_df.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,284807.0,142404.00,82216.84,1.00,71202.50,142404.00,213605.50,284807.00
Time,284807.0,94813.86,47488.15,0.00,54201.50,84692.00,139320.50,172792.00
V1,284807.0,0.00,1.96,-56.41,-0.92,0.02,1.32,2.45
V2,284807.0,0.00,1.65,-72.72,-0.60,0.07,0.80,22.06
V3,284807.0,-0.00,1.52,-48.33,-0.89,0.18,1.03,9.38
V4,284807.0,0.00,1.42,-5.68,-0.85,-0.02,0.74,16.88
V5,284807.0,0.00,1.38,-113.74,-0.69,-0.05,0.61,34.80
V6,284807.0,0.00,1.33,-26.16,-0.77,-0.27,0.40,73.30
V7,284807.0,-0.00,1.24,-43.56,-0.55,0.04,0.57,120.59
V8,284807.0,0.00,1.19,-73.22,-0.21,0.02,0.33,20.01


In [42]:
sc = StandardScaler()
credit_df[[ 'Amount' ]]  =  sc.fit_transform(credit_df[['Amount']])

# check data balance

In [43]:
credit_df["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [44]:
credit_df["Class"].value_counts(normalize=True)*100

0    99.827251
1     0.172749
Name: Class, dtype: float64

## remove unnecessary columns

In [45]:
#remove columns that are not required
credit_df = credit_df.drop(columns=["Time","Unnamed: 0", "Amount"])

#pop and store target separately
target = credit_df.pop("Class")

#whatever columns are left are ALL FEATURES
features = credit_df

In [47]:
len(features.columns)

28

In [49]:
pca_model = PCA(n_components=15)

pca_features = pca_model.fit_transform(features)



In [51]:
names = [f"PC_{count}" for count in range(1,16,1)]

features = pd.DataFrame(pca_features, columns=names)

features.head(3)

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13,PC_14,PC_15
0,1.359801,0.072781,-2.536356,1.378162,0.338325,0.462445,0.239458,-0.098564,0.363789,-0.090855,-0.551614,0.617728,-0.991398,0.311653,1.468460
1,-1.191836,-0.266144,-0.166447,0.448136,-0.060027,-0.082572,-0.078317,-0.085543,-0.255459,0.167190,1.612576,-1.065164,0.489161,0.142143,0.634632
2,1.358351,1.340162,-1.773212,0.379781,0.503198,1.800519,0.791423,-0.247652,-1.514650,-0.207657,0.624521,-0.066064,0.717276,0.166083,2.346262


In [53]:
from sklearn.metrics import classification_report
def output(models):
    for model in models:
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=10, stratify=target)
        clf = model.fit(X_train, y_train)
        predicted_values = model.predict(X_test)
        print(classification_report(y_test, predicted_values))

In [54]:
logit_model = LogisticRegression()
dtree_model = DecisionTreeClassifier()
svc_model = SVC()

output([logit_model, dtree_model, svc_model])

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.85      0.54      0.66        98

    accuracy                           1.00     56962
   macro avg       0.93      0.77      0.83     56962
weighted avg       1.00      1.00      1.00     56962

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.74      0.75        98

    accuracy                           1.00     56962
   macro avg       0.88      0.87      0.88     56962
weighted avg       1.00      1.00      1.00     56962

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.95      0.70      0.81        98

    accuracy                           1.00     56962
   macro avg       0.97      0.85      0.90     56962
weighted avg       1.00      1.00      1.00     56962



# Synthetic Minority Oversampling Technique (SMOTE)

A problem with imbalanced classification is that there are too few examples of the minority class for a model to effectively learn the decision boundary.

One way to solve this problem is to oversample the examples in the minority class. Perhaps the most widely used approach to synthesizing new examples is called the Synthetic Minority Oversampling TEchnique, or SMOTE for short. This technique was described by Nitesh Chawla, et al. in their 2002 paper named for the technique titled “SMOTE: Synthetic Minority Over-sampling Technique.”

In [55]:
#SMOTE

from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=42)

X = features
y = target

X_sm,y_sm = sm.fit_resample(X,y)

In [56]:
pd.Series(y_sm).value_counts()

0    284315
1    284315
Name: Class, dtype: int64

### Train the model on the new dataset with synthetic points added

### BONUS: We can test the model on the new set as well for sake of comparison

In [60]:
#write logic for training on enhanced data & testing on orginal values in one common function
#run the logic in a loop for 3 models we are creating

def smote_train_test(  models : list, features_original, target_orginal, features_enhanced, target_enhanced ):

    #repeat the process for each model in models list    
    for model in models:

        #split enhanced data
        X_train, X_test, y_train, y_test = train_test_split(features_enhanced, target_enhanced, test_size=0.2, random_state=42)

        #train the model on enhanced data
        model.fit(X_train, y_train)

        #split original data
        X_train, X_test, y_train, y_test = train_test_split(features_original, target_orginal, test_size=0.2, random_state=42)

        #make prediction on original data
        predicted_values = model.predict(X_test)

        #print classification report for tests performed on original data
        print(classification_report(y_test, predicted_values))


In [ ]:
condition = (    credit_df['Class'] == 1   )
credit_df[   condition   ]

In [62]:
logit_model = LogisticRegression()
dtree_model = DecisionTreeClassifier()
svc_model = SVC()

smote_train_test([logit_model, dtree_model, svc_model],  X, y, X_sm, y_sm )

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.93      0.11        98

    accuracy                           0.98     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.98      0.99     56962

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.70      1.00      0.82        98

    accuracy                           1.00     56962
   macro avg       0.85      1.00      0.91     56962
weighted avg       1.00      1.00      1.00     56962

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.09      0.95      0.17        98

    accuracy                           0.98     56962
   macro avg       0.55      0.97      0.58     56962
weighted avg       1.00      0.98      0.99     56962



### Now testing the model on the original dataset for actual benchmark

ENSEMBLE LEARNING:

Random Forest model : multiple dtree models used collectively
Gradient Boosting
ADABoosting
XGBoost


There are 2 types of ensembles:

100 dtrees used in an ensemble

a) Bagging : we ask for an answer from all 100 models and go with majority
b) Boosted : we ask for an answer from the first model. The misclassification from the first
                model is sent as input to the second model that tries to correct the mistake
                This process of "make n+1 model to fix misclassification of nth model"


